In [24]:
#%pip install pystardog
#%pip install dotenv
#%pip install Unidecode
#%pip install langdetect
#%pip install -U pip setuptools wheel
#%pip install -U spacy
#!python -m spacy download en_core_web_trf


In [52]:
import stardog
import os
import pandas as pd
import io
from unidecode import unidecode
import re
from langdetect import detect
import spacy
import requests, uuid, json
#from dotenv import load_dotenv

### Specify Stardog connection details

In [3]:
# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

### Create a new database in Stardog

Drop the database if it already exists.

In [4]:
#database_name = 'IndigenousSlavery'

#with stardog.Admin(**connection_details) as admin:
#   if database_name in [db.name for db in admin.databases()]:
#       admin.database(database_name).drop()
#   db = admin.new_database(database_name)

### Add the database schema

In [5]:
####
# Opcao - Adicionar o schema via Stardog Studio

### Add data to the database

In [6]:
# Lendo os dados do arquivo CSV
data = pd.read_csv("data/BDTD_Consolidado.csv")
data.head()

,Autor(a),ID Lattes do(a) autor(a),Orientadores,ID Lattes dos orientadores,Membros da banca,ID Lattes dos membros da banca,Título,Ano de defesa,Instituição de defesa,Sigla da instituição de defesa,...,Tipos de acesso,Tipo de documento,Assuntos em português,Assuntos em inglês,Idioma,Resumo,Link de acesso,Resumo em Português,Resumo em Inglês,Referência Bibliográfica
0,"Krob, Bruna Emerim",Não informado pela instituição,"['Xavier, Regina Célia Lima']",Não informado pela instituição,[],Não informado pela instituição,"""Com a condição de servir gratuitamente a mim ...",2016,Universidade Federal do Rio Grande do Sul (UFRGS),UFRGS,...,openAccess,masterThesis,"['Escravidão', 'Abolição da escravatura', 'Car...","['Slavery', 'Manumission', 'Abolition', 'Freed...",por,A presente pesquisa parte da emancipação de es...,http://hdl.handle.net/10183/148467,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição
1,"Brusantin, Beatriz de Miranda",Não informado pela instituição,[],Não informado pela instituição,[],Não informado pela instituição,Capitães e Mateus : relações sociais e as cult...,2011,Universidade Estadual de Campinas (UNICAMP),UNICAMP,...,openAccess,doctoralThesis,"['Escravidão - Pernambuco', 'Escravos - Alforr...",['Não informado pela instituição'],por,Orientador: Robert Wayne Andrew Slenes,https://hdl.handle.net/20.500.12733/1615341||B...,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição
2,"Palma, Rogério da",Não informado pela instituição,"['Truzzi, Oswaldo Mário Serra']",http://lattes.cnpq.br/5005536921435787||Não in...,[],Não informado pela instituição,Liberdade sob tensão: negros e relações interp...,2014,Universidade Federal de São Carlos (UFSCAR),UFSCAR,...,openAccess,doctoralThesis,"['Negros', 'Escravidão - pós-abolição', 'Relaç...","['Blacks', 'Post-abolition', 'Interpersonal re...",por,Post-abolition can be conceptualized as a hist...,https://repositorio.ufscar.br/handle/ufscar/6690,Não informado pela instituição,Não informado pela instituição,"PALMA, Rogério da. Liberdade sob tensão: negro..."
3,"Sgarbosa, Lívia",Não informado pela instituição,"['Nascimento, Manoel Nelito Matheus']",http://lattes.cnpq.br/4486910967478337||Não in...,[],Não informado pela instituição,O Colégio São Benedito e a escolarização da po...,2018,Universidade Federal de São Carlos (UFSCAR),UFSCAR,...,openAccess,masterThesis,"['Educação de negros', 'Colégio São Benedito',...","['Black education', 'Post-abolition']",por,In the post-abolition period and in the First ...,https://repositorio.ufscar.br/handle/ufscar/11302,Não informado pela instituição,Não informado pela instituição,"SGARBOSA, Lívia. O Colégio São Benedito e a es..."
4,"Alcântara, Mauro Henrique Miranda de",Não informado pela instituição,[],Não informado pela instituição,[],Não informado pela instituição,D. Pedro II e a emancipação dos escravos,2013,Universidade Federal de Mato Grosso (UFMT),UFMT,...,openAccess,masterThesis,"['D. Pedro II', 'Abolição da escravidão', 'Bra...",['Não informado pela instituição'],por,When you mention the topic abolition of slaver...,"ALCÂNTARA, Mauro Henrique Miranda de. D. Pedro...",Não informado pela instituição,Não informado pela instituição,Não informado pela instituição


Preparando o texto em turtle

In [7]:
# Prefixos
prefixos = """ @prefix ns: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix event: <http://purl.org/NET/c4dm/event.owl#> .
@prefix prism: <http://prismstandard.org/namespaces/1.2/basic/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schemas.talis.com/2005/address/schema#> .
@prefix status: <http://purl.org/ontology/bibo/status/> .
@prefix degrees: <http://purl.org/ontology/bibo/degrees/> .
@prefix stardog: <tag:stardog:api:> .
@base <http://www.w3.org/2002/07/owl#> .

"""

Conectando com o banco e adicionando as triplas

In [8]:
def add_triplas_to_stardog(prefixos, triplas):

    # Incluindo prefixos às triplas
    triplas = prefixos + " " + triplas

    ### Connect to the Stardog database
    database_name = 'IndigenousSlavery'
    conn = stardog.Connection(database_name, **connection_details)

    conn.begin()
    conn.add(stardog.content.Raw(triplas, 'text/turtle'))
    conn.commit() # commit the transaction

Limpando e processando os dados das Teses

In [39]:
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))

In [106]:
titulo = data['Título']
thesis_uri = data['Título'].apply(lambda x: process_uri(x))

In [13]:
#Alterando os anos com dois dígitos para 4 dígitos
data['Ano de defesa'][data['Ano de defesa'].apply(str).apply(len) != 4] = data['Ano de defesa'][data['Ano de defesa'].apply(str).apply(len) != 4].apply(lambda x: 2000 + x if x < 25 else 1900 + x)
ano = data['Ano de defesa']

C:\Users\facordei\AppData\Local\Temp\ipykernel_17376\1699272387.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['Ano de defesa'][data['Ano de defesa'].apply(str).apply(len) != 4] = data['Ano de defesa'][data['Ano de defesa'].apply(s

In [14]:
link = data['Link de acesso']
referencia = data['Referência Bibliográfica']
resumo = data['Resumo']
resumo_pt = data['Resumo em Português']
resumo_en = data['Resumo em Inglês']
resumo_lang = data['Resumo'].apply(lambda x: detect(str(x) + "sem texto"))


In [16]:
triplas = """ """

for m in range(len(thesis_uri)):
    tripla = """
    stardog:""" + thesis_uri[m] + """ rdf:type owl:NamedIndividual.
    stardog:""" + thesis_uri[m] + """ rdf:type bibo:Thesis.
    stardog:""" + thesis_uri[m] + """ terms:title '""" + titulo[m].replace("'","").replace(u'\\', u' ') +  """'.
    stardog:""" + thesis_uri[m] + """ terms:created '""" + str(ano[m]) +  """'^^xsd:gYear.
    """
    if link[m].startswith("http"):
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:repository '""" + link[m].replace("'","").replace(u'\\', u' ') + """'. """

    if referencia[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:identifier '""" + referencia[m].replace("'","").replace(u'\\', u' ') + """'. """

    if resumo[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo[m]).replace("'","").replace(u'\\', u' ') + """'@""" + resumo_lang[m] + """. """
    
    if resumo_pt[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo_pt[m]).replace("'","").replace(u'\\', u' ') + """'@pt. """

    if resumo_en[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo_en[m]).replace("'","").replace(u'\\', u' ') + """'@en. """

    triplas = triplas + " " + tripla

In [ ]:
add_triplas_to_stardog(prefixos, triplas)

Limpando dados e propriedades das pessoas ("Person") - Autores, Orientadores e Membros da Banca.

In [9]:
def clean_person(person_column):

    column_raw = person_column.apply(lambda x: x.split("',")).replace("['", "")

    column_raw_list = []
    for i in column_raw:
        for person in i:
            if person != '[]':
                person = person.replace("[", "").replace("]", "").replace("'", "").replace("", "").split(",")
                column_raw_list.append(person)

    column_raw_list = pd.Series(column_raw_list) 

    #Último Sobrenome   
    column_family_name = column_raw_list.apply(lambda x: x[0].strip().replace(u'\\', u' '))

    # Primeiros nomes
    column_given_name = column_raw_list.apply(lambda x: x[1].strip().replace(u'\\', u' ') if len(x) > 1 else "")

    # Unindo nome e sobrenome
    column_uri = (column_given_name + "_" + column_family_name) 

    #substituindo espaco por "_", retirando diacrítico e colocando em minúscula
    column_uri = column_uri.apply(lambda x: process_uri(x))

    return (column_raw_list, column_family_name, column_given_name, column_uri)  

In [ ]:
# Processando colunas
autor_raw_list, autor_family_name, autor_given_name, autor_uri = clean_person(data["Autor(a)"])
orientador_raw_list, orientador_family_name, orientador_given_name, orientador_uri = clean_person(data['Orientadores'])
banca_raw_list, banca_family_name, banca_given_name, banca_uri = clean_person(data['Membros da banca'])

# Unindo as colunas com informaoes de pessoas
family_name = pd.concat([autor_family_name, orientador_family_name, banca_family_name],  ignore_index=True)
given_name = pd.concat([autor_given_name, orientador_given_name, banca_given_name],  ignore_index=True)
name_uri = pd.concat([autor_uri, orientador_uri, banca_uri],  ignore_index=True)

# Removendo duplicados
name_uri = name_uri.drop_duplicates()
uniq_index = name_uri.index
family_name = family_name[uniq_index]
given_name = given_name[uniq_index]

In [17]:
triplas = """ """

for m in uniq_index:
    tripla = """
    stardog:""" + name_uri[m] + """ rdf:type owl:NamedIndividual.
    stardog:""" + name_uri[m] + """ rdf:type foaf:Person.
    stardog:""" + name_uri[m] + """ rdfs:label '""" + family_name[m].replace("'","") + """, """+ given_name[m].replace("'","") + """'.
    stardog:""" + name_uri[m] + """ foaf:givenname '""" + given_name[m].strip().replace("'","") + """'.
    stardog:""" + name_uri[m] + """ foaf:family_name '""" + family_name[m].replace("'","") + """'.
    """

    triplas = triplas + " " + tripla

In [ ]:
add_triplas_to_stardog(prefixos, triplas)

In [18]:
# Adding autor relationship between thesis and its author.

triplas = """ """

for m in range(len(thesis_uri)):
    tripla = """
    stardog:""" + thesis_uri[m] + """ bibo:author stardog:""" + autor_uri[m] +  """."""
    triplas = triplas + " " + tripla

In [19]:
add_triplas_to_stardog(prefixos, triplas)

In [68]:
# Adding autor relationship between thesis and its advisors and opponents.

triplas = """ """

for m in range(len(thesis_uri)):
    person_uris = clean_person(data['Orientadores'][m:m+1])[3]
    for person_uri in person_uris:
        tripla = """
        stardog:""" + thesis_uri[m] + """ bibo:advisor stardog:""" + person_uri +  """."""
        triplas = triplas + " " + tripla

    person_uris = clean_person(data['Membros da banca'][m:m+1])[3]
    for person_uri in person_uris:
        tripla = """
        stardog:""" + thesis_uri[m] + """ bibo:opponent stardog:""" + person_uri +  """."""
        triplas = triplas + " " + tripla


In [69]:
add_triplas_to_stardog(prefixos, triplas)

Limpando dados e propriedades da instituicao de pesquisa

In [98]:
#Criando as triplas das instituicões
university = data[['Instituição de defesa', 'Sigla da instituição de defesa']].drop_duplicates(ignore_index=True)

triplas = """ """

for m in range(len(university)):
    tripla = """
    stardog:""" + university['Sigla da instituição de defesa'][m] + """ rdf:type owl:NamedIndividual.
    stardog:""" + university['Sigla da instituição de defesa'][m] + """ rdf:type bibo:University. 
    stardog:""" + university['Sigla da instituição de defesa'][m] + """ rdfs:label '""" + university['Instituição de defesa'][m] + """'.
    stardog:""" + university['Sigla da instituição de defesa'][m] + """ bibo:acronym '""" + university['Sigla da instituição de defesa'][m] + """'.
    """
    triplas = triplas + " " + tripla

In [101]:
add_triplas_to_stardog(prefixos, triplas)

In [135]:
# Criando as triplas dos departamentos e programas
departamento = data[data['Departamento da instituição de defesa'] != 'Não informado pela instituição']
departamento = departamento[['Sigla da instituição de defesa', 'Departamento da instituição de defesa']].drop_duplicates(ignore_index=True)
departamento['uri'] = departamento['Sigla da instituição de defesa'] + '_' + departamento['Departamento da instituição de defesa'].apply(process_uri)


programa = data[data['Programa de Pós-Graduação da instituição de defesa'] != 'Não informado pela instituição']
programa = programa[['Sigla da instituição de defesa', 'Programa de Pós-Graduação da instituição de defesa']].drop_duplicates(ignore_index=True)
programa['uri'] = programa['Sigla da instituição de defesa'] + '_' + programa['Programa de Pós-Graduação da instituição de defesa'].apply(process_uri)

triplas = """ """

for m in range(len(departamento)):
    tripla = """
    stardog:""" + departamento['uri'][m] + """ rdf:type owl:NamedIndividual.
    stardog:""" + departamento['uri'][m] + """ rdf:type bibo:Department. 
    stardog:""" + departamento['uri'][m] + """ rdfs:label '""" + departamento['Departamento da instituição de defesa'][m] + """'.
    stardog:""" + departamento['uri'][m] + """ <http://purl.obolibrary.org/obo/BFO_0000050> stardog:""" + departamento['Sigla da instituição de defesa'][m] + """.
    """
    triplas = triplas + " " + tripla

for m in range(len(programa)):
    tripla = """
    stardog:""" + programa['uri'][m] + """ rdf:type owl:NamedIndividual.
    stardog:""" + programa['uri'][m] + """ rdf:type bibo:Program. 
    stardog:""" + programa['uri'][m] + """ rdfs:label '""" + programa['Programa de Pós-Graduação da instituição de defesa'][m] + """'.
    stardog:""" + programa['uri'][m] + """ <http://purl.obolibrary.org/obo/BFO_0000050> stardog:""" + programa['Sigla da instituição de defesa'][m] + """.
    """
    triplas = triplas + " " + tripla

In [136]:
add_triplas_to_stardog(prefixos, triplas)

Conectando as teses às organizacoes (univeridades, departamentos e programas)

In [141]:
publishers = data[['Sigla da instituição de defesa', 'Departamento da instituição de defesa', 'Programa de Pós-Graduação da instituição de defesa']] 
publishers['dep_uri'] = publishers['Sigla da instituição de defesa'] + '_' + publishers['Departamento da instituição de defesa'].apply(process_uri)
publishers['program_uri'] = publishers['Sigla da instituição de defesa'] + '_' + publishers['Programa de Pós-Graduação da instituição de defesa'].apply(process_uri)


triplas = """ """

for m in range(len(thesis_uri)):
    if publishers['Sigla da instituição de defesa'][m] != 'Não informado pela instituição':
        tripla = """
        stardog:""" + thesis_uri[m] + """ terms:publisher stardog:""" + publishers['Sigla da instituição de defesa'][m] +  """."""
        triplas = triplas + " " + tripla

    if publishers['Departamento da instituição de defesa'][m] != 'Não informado pela instituição':
        tripla = """
        stardog:""" + thesis_uri[m] + """ terms:publisher stardog:""" + publishers['dep_uri'][m] +  """."""
        triplas = triplas + " " + tripla

    if publishers['Programa de Pós-Graduação da instituição de defesa'][m] != 'Não informado pela instituição':
        tripla = """
        stardog:""" + thesis_uri[m] + """ terms:publisher stardog:""" + publishers['program_uri'][m] +  """."""
        triplas = triplas + " " + tripla

C:\Users\facordei\AppData\Local\Temp\ipykernel_17376\1663232402.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publishers['dep_uri'] = publishers['Sigla da instituição de defesa'] + '_' + publishers['Departamento da instituição de defesa'].apply(process_uri)
C:\Users\facordei\AppData\Local\Temp\ipykernel_17376\1663232402.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publishers['program_uri'] = publishers['Sigla da instituição de defesa'] + '_' + publishers['Programa de Pós-Graduação da instituiçã

In [142]:
add_triplas_to_stardog(prefixos, triplas)

Limpando os assuntos e áreas do conhecimento

In [35]:
# Filtrando as listas unicas (SET) com assuntos
# Inglês
assunto_en_v1 = data['Assuntos em inglês'].apply(lambda x: x.replace("'", "").replace("[", "").replace("]", "").split(",")).values

assunto_en_v2 = []
for i in assunt_en_v1:
    assunto_en_v2 = assunto_en_v2 + i

assunto_en_v3 = []
for i in assunto_en_v2:
    assunto_en_v3.append(i.strip())
assunto_en = set(assunto_en_v3)

#Português
assunto_pt_v1 = data['Assuntos em português'].apply(lambda x: x.replace("'", "").replace("[", "").replace("]", "").split(",")).values

assunto_pt_v2 = []
for i in assunto_pt_v1:
    assunto_pt_v2 = assunto_pt_v2 + i

assunto_pt_v3 = []
for i in assunto_pt_v2:
    assunto_pt_v3.append(i.strip())
assunto_pt = set(assunto_pt_v3)

In [44]:
# Funcao para encontrar o lemma da palavra e formar a URI do assunto

nlp = spacy.load("en_core_web_trf")

def lemma(word):
#    ex = ['Political philosophy','Legal form','Penal abolitionism','Marxism','Criminology']
    doc = nlp(word)
    uri = ''
    for token in doc:
        uri = uri + token.lemma_ + ' '
    return (process_uri(uri))

In [55]:
# Funcao para traduzir os assuntos em português para inglês

# Add your key and endpoint
key = os.getenv('TRANSLATOR_KEY') #"<your-translator-key>"
endpoint = os.getenv('TRANSLATOR_ENDPOINT') #"https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = os.getenv('TRANSLATOR_LOCATION') #"<YOUR-RESOURCE-LOCATION>"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'pt',
    'to': ['en']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
#body = [{
#    'text': 'I would really like to drive your car around the block a few times!'
#}]

def translate_pt_en(word):
    body = [{'text': word}]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    return (response[0]['translations'][0]['text'])
    #print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))

In [66]:
# Criando um dicionário com assunto_URI e termos em inglês
uri_dic = {}

for word in assunto_en:
    assunto_uri = lemma(word)
    if assunto_uri not in uri_dic:
        uri_dic[assunto_uri] = {'en': [word], 'pt': []}
    else:
        uri_dic[assunto_uri]['en'].append(word)

In [67]:
# Criando um dicionário com assunto_URI e termos em inglês

for word in assunto_pt:
    assunto_translate = translate_pt_en(word)
    assunto_uri = lemma(assunto_translate)
    if assunto_uri not in uri_dic:
        uri_dic[assunto_uri] = {'en': [assunto_translate], 'pt': [word]}
    else:
        uri_dic[assunto_uri]['en'] = list(set(uri_dic[assunto_uri]['en'] + [assunto_translate]))
        uri_dic[assunto_uri]['pt'].append(word)


In [ ]:
uri_dic

{'': {'en': [''], 'pt': ['']},
 'city': {'en': ['Cities', 'City'], 'pt': ['Cities']},
 'of_attention_deficit_disorder_with_hyperactivity_of': {'en': ['Of attention deficit disorder with hyperactivity of'],
  'pt': []},
 'role_model': {'en': ['Role model'], 'pt': []},
 'family': {'en': ['Family', 'family'], 'pt': []},
 'genotyping': {'en': ['Genotyping'], 'pt': []},
 'production_of_black_knowledge': {'en': ['Production of Black Knowledge'],
  'pt': []},
 'rabble': {'en': ['Rabble'], 'pt': []},
 'countryside_education': {'en': ['Countryside education',
   'Countryside Education'],
  'pt': []},
 'personal_narrative': {'en': ['Personal narratives'], 'pt': []},
 'african_brazilian_study_groups': {'en': ['African Brazilian Study Groups'],
  'pt': []},
 'history_teaching': {'en': ['history teaching',
   'History teaching',
   'History Teaching'],
  'pt': []},
 'criminal_subjection': {'en': ['Criminal subjection'], 'pt': []},
 'totalitarianism': {'en': ['Totalitarianism'], 'pt': []},
 'land_pr

In [ ]:
# Carregando as triplas dos assuntos na base de dados 

for uri in uri_dic:
    tripla = """
    stardog:""" + uri + """ rdf:type bibo:Subject."""
    triplas = triplas + " " + tripla

    for label_pt in uri_dic[uri]['pt']:
        tripla = tripla + """stardog:""" + uri + """ rdfs:label '""" + str(label_pt).replace("'","").replace(u'\\', u' ') + """'@pt. """

    for label_en in uri_dic[uri]['en']:
        tripla = tripla + """stardog:""" + uri + """ rdfs:label '""" + str(label_en).replace("'","").replace(u'\\', u' ') + """'@en. """

In [ ]:
#add_triplas_to_stardog(prefixos, triplas)

In [10]:
ex = data['Assuntos em inglês'][0].replace("'", "").replace("[", "").replace("]", "").split(",")
ex[0]
ex_dic = []
for i in ex:
    ex_dic.append({'text': i})

ex_dic


[{'text': 'Slavery'},
 {'text': ' Manumission'},
 {'text': ' Abolition'},
 {'text': ' Freedom'}]

In [12]:
import requests, uuid, json

# Add your key and endpoint
key = os.getenv('TRANSLATOR_KEY') #"<your-translator-key>"
endpoint = os.getenv('TRANSLATOR_ENDPOINT') #"https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = os.getenv('TRANSLATOR_LOCATION') #"<YOUR-RESOURCE-LOCATION>"

path = '/translate'
constructed_url = endpoint + path

In [15]:
params = {
    'api-version': '3.0',
    'from': 'en',
    'to': ['pt']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
#body = [{
#    'text': 'I would really like to drive your car around the block a few times!'
#}]
body = ex_dic

request = requests.post(constructed_url, params=params, headers=headers, json=body)
response = request.json()

print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))

[
    {
        "translations": [
            {
                "text": "Escravidão",
                "to": "pt"
            }
        ]
    },
    {
        "translations": [
            {
                "text": " Manumission",
                "to": "pt"
            }
        ]
    },
    {
        "translations": [
            {
                "text": " Abolição",
                "to": "pt"
            }
        ]
    },
    {
        "translations": [
            {
                "text": " Liberdade",
                "to": "pt"
            }
        ]
    }
]


In [16]:
response

[{'translations': [{'text': 'Escravidão', 'to': 'pt'}]},
 {'translations': [{'text': ' Manumission', 'to': 'pt'}]},
 {'translations': [{'text': ' Abolição', 'to': 'pt'}]},
 {'translations': [{'text': ' Liberdade', 'to': 'pt'}]}]

In [18]:
import spacy
from spacy.lang.en.examples import sentences 
ex = ['Political philosophy','Legal form','Penal abolitionism','Marxism','Criminology']
nlp = spacy.load("en_core_web_trf")
for word in ex:
    doc = nlp(word)
    print(doc.text)
    for token in doc:
        print(token.lemma_)

Political philosophy
political
philosophy
Legal form
legal
form
Penal abolitionism
penal
abolitionism
Marxism
marxism
Criminology
criminology


### Clean up the connection

Normally you would use a with statement similar to line 3.

In [9]:
conn.__exit__()

In [ ]:
query = """
SELECT ?m ?r ?n
WHERE {?m ?r ?n.
}
LIMIT 150
"""

csv_results = conn.select(query, content_type='text/csv')
df = pd.read_csv(io.BytesIO(csv_results))
df.head()